In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
%matplotlib inline

In [10]:
df = pd.read_csv('df.csv',index_col=0)
df.head(10)

,match_id,time,gold,top_gold,jgl_gold,mid_gold,adc_gold,sup_gold,kill_total,assist_total,opp_kill_total,opp_assist_total,r_inhib_count,r_baron_count,r_tower_count,r_herald_count,r_drag_count,b_inhib_count,b_baron_count,b_tower_count,b_herald_count,b_dragon_count,kda,opp_kda,result
0,001a1f289e3bab22_b,0.0,2415.0,475.0,475.0,475.0,475.0,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,001a1f289e3bab22_b,1.0,2430.0,475.0,475.0,475.0,475.0,530.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,001a1f289e3bab22_b,2.0,2751.0,532.0,552.0,532.0,532.0,603.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,001a1f289e3bab22_b,3.0,3929.0,849.0,817.0,791.0,686.0,786.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,001a1f289e3bab22_b,4.0,5219.0,1099.0,1099.0,1107.0,976.0,938.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
5,001a1f289e3bab22_b,5.0,6418.0,1466.0,1278.0,1352.0,1196.0,1126.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.00,1.5,0.0
6,001a1f289e3bab22_b,6.0,8156.0,1667.0,1708.0,1574.0,1687.0,1520.0,2.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,3.5,0.0
7,001a1f289e3bab22_b,7.0,9520.0,1970.0,2097.0,1708.0,1995.0,1750.0,2.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.00,3.5,0.0
8,001a1f289e3bab22_b,8.0,10681.0,2147.0,2414.0,1988.0,2244.0,1888.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.75,4.5,0.0
9,001a1f289e3bab22_b,9.0,12036.0,2605.0,2533.0,2321.0,2535.0,2042.0,2.0,1.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.75,4.5,0.0


In [11]:
df.shape

(286876, 25)

## Fit Scaler to Full Dataset


In [12]:
# this df contains 2 games worth of rows
dft = df.head(76).copy()

In [13]:
# this df contains 1 game worth of rows
dfs = df.head(38).copy()

In [15]:
from sklearn.preprocessing import StandardScaler

data = df.iloc[:,2:-1]
scaler = StandardScaler()
scaler.fit(data)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [16]:
df.head()

,match_id,time,gold,top_gold,jgl_gold,mid_gold,adc_gold,sup_gold,kill_total,assist_total,opp_kill_total,opp_assist_total,r_inhib_count,r_baron_count,r_tower_count,r_herald_count,r_drag_count,b_inhib_count,b_baron_count,b_tower_count,b_herald_count,b_dragon_count,kda,opp_kda,result
0,001a1f289e3bab22_b,0.0,2415.0,475.0,475.0,475.0,475.0,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,001a1f289e3bab22_b,1.0,2430.0,475.0,475.0,475.0,475.0,530.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,001a1f289e3bab22_b,2.0,2751.0,532.0,552.0,532.0,532.0,603.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,001a1f289e3bab22_b,3.0,3929.0,849.0,817.0,791.0,686.0,786.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,001a1f289e3bab22_b,4.0,5219.0,1099.0,1099.0,1107.0,976.0,938.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Split into Train/Test Sets and Standardize

In [17]:
cols = list(df.columns.values)
hold_out_cols = ['match_id', 'time', 'result']
ho_df = df[hold_out_cols].copy()

In [18]:
cols_scale = ['gold', 'top_gold', 'jgl_gold', 'mid_gold', 'adc_gold', 'sup_gold', 'kill_total',
 'assist_total', 'opp_kill_total', 'opp_assist_total', 'r_inhib_count', 'r_baron_count', 'r_tower_count',
 'r_herald_count', 'r_drag_count', 'b_inhib_count', 'b_baron_count', 'b_tower_count', 'b_herald_count',
 'b_dragon_count', 'kda', 'opp_kda']

In [19]:
data = scaler.transform(data)

In [20]:
sdf = pd.DataFrame(data)
sdf.columns = cols_scale
sdf.head()

,gold,top_gold,jgl_gold,mid_gold,adc_gold,sup_gold,kill_total,assist_total,opp_kill_total,opp_assist_total,r_inhib_count,r_baron_count,r_tower_count,r_herald_count,r_drag_count,b_inhib_count,b_baron_count,b_tower_count,b_herald_count,b_dragon_count,kda,opp_kda
0,-1.342942,-1.326213,-1.387083,-1.341606,-1.320149,-1.269691,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698
1,-1.342244,-1.326213,-1.387083,-1.341606,-1.320149,-1.264782,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698
2,-1.327294,-1.313755,-1.368286,-1.330031,-1.308748,-1.240890,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698
3,-1.272434,-1.244472,-1.303596,-1.277439,-1.277944,-1.180998,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698
4,-1.212358,-1.189832,-1.234757,-1.213273,-1.219937,-1.131251,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698


In [21]:
print(len(sdf))
print(len(ho_df))

286876
286876


In [22]:
sdf = pd.merge(sdf, ho_df, left_index=True, right_index=True)
sdf.head()

,gold,top_gold,jgl_gold,mid_gold,adc_gold,sup_gold,kill_total,assist_total,opp_kill_total,opp_assist_total,r_inhib_count,r_baron_count,r_tower_count,r_herald_count,r_drag_count,b_inhib_count,b_baron_count,b_tower_count,b_herald_count,b_dragon_count,kda,opp_kda,match_id,time,result
0,-1.342942,-1.326213,-1.387083,-1.341606,-1.320149,-1.269691,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698,001a1f289e3bab22_b,0.0,0.0
1,-1.342244,-1.326213,-1.387083,-1.341606,-1.320149,-1.264782,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698,001a1f289e3bab22_b,1.0,0.0
2,-1.327294,-1.313755,-1.368286,-1.330031,-1.308748,-1.240890,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698,001a1f289e3bab22_b,2.0,0.0
3,-1.272434,-1.244472,-1.303596,-1.277439,-1.277944,-1.180998,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698,001a1f289e3bab22_b,3.0,0.0
4,-1.212358,-1.189832,-1.234757,-1.213273,-1.219937,-1.131251,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698,001a1f289e3bab22_b,4.0,0.0


In [23]:
fcols = [ 'match_id', 'time','gold', 'top_gold', 'jgl_gold', 'mid_gold', 'adc_gold', 'sup_gold', 'kill_total',
 'assist_total', 'opp_kill_total', 'opp_assist_total', 'r_inhib_count', 'r_baron_count', 'r_tower_count',
 'r_herald_count', 'r_drag_count', 'b_inhib_count', 'b_baron_count', 'b_tower_count', 'b_herald_count',
 'b_dragon_count', 'kda', 'opp_kda', 'result']

In [133]:
sdf1 = sdf[fcols]
sdf1.head()

,match_id,time,gold,top_gold,jgl_gold,mid_gold,adc_gold,sup_gold,kill_total,assist_total,opp_kill_total,opp_assist_total,r_inhib_count,r_baron_count,r_tower_count,r_herald_count,r_drag_count,b_inhib_count,b_baron_count,b_tower_count,b_herald_count,b_dragon_count,kda,opp_kda,result
0,001a1f289e3bab22_b,0.0,-1.342942,-1.326213,-1.387083,-1.341606,-1.320149,-1.269691,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698,0.0
1,001a1f289e3bab22_b,1.0,-1.342244,-1.326213,-1.387083,-1.341606,-1.320149,-1.264782,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698,0.0
2,001a1f289e3bab22_b,2.0,-1.327294,-1.313755,-1.368286,-1.330031,-1.308748,-1.240890,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698,0.0
3,001a1f289e3bab22_b,3.0,-1.272434,-1.244472,-1.303596,-1.277439,-1.277944,-1.180998,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698,0.0
4,001a1f289e3bab22_b,4.0,-1.212358,-1.189832,-1.234757,-1.213273,-1.219937,-1.131251,-0.899995,-0.796908,-0.899995,-0.796908,-1.742076,-2.271392,-1.401829,-2.768166,-1.835765,-1.735504,-2.43541,-1.461121,-2.877854,-1.885778,-0.712698,-0.712698,0.0


In [161]:
len(sdf1)

286844

In [162]:
X = sdf1.loc[:,sdf1.columns != 'result'].values
y = sdf1.loc[:,sdf1.columns == 'result'].values
groups = sdf1['match_id']

In [163]:
from sklearn.model_selection import GroupKFold
gkf = GroupKFold(n_splits=2)

for train_index, test_index in gkf.split(X, y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

### Prepare Train Data

In [164]:
xtrain = pd.DataFrame(X_train)
ytrain = pd.DataFrame(y_train)

In [165]:
print(len(ytrain))
print(len(xtrain))

143419
143419


In [166]:
train_df = pd.merge(xtrain, ytrain, left_index=True, right_index=True)

In [167]:
train_df.rename(columns={'0_y':'result','0_x':'match_id'},inplace=True)

In [168]:
train_df.sort_values(['match_id',1],inplace=True)
train_df.head()

,match_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,result
0,001a1f289e3bab22_r,0,-1.34294,-1.32621,-1.38708,-1.34161,-1.32015,-1.26969,-0.899995,-0.796908,-0.899995,-0.796908,-0.865652,-2.27139,0.90352,-2.76817,1.59489,-1.7355,-2.43541,-0.500126,-2.87785,-1.00394,-0.712698,-0.712698,1.0
1,001a1f289e3bab22_r,1,-1.34294,-1.32621,-1.38708,-1.34161,-1.32015,-1.26969,-0.899995,-0.796908,-0.899995,-0.796908,-0.865652,-2.27139,0.90352,-2.76817,1.59489,-1.7355,-2.43541,-0.500126,-2.87785,-1.00394,-0.712698,-0.712698,1.0
2,001a1f289e3bab22_r,2,-1.32944,-1.31376,-1.37317,-1.33003,-1.30875,-1.2494,-0.899995,-0.796908,-0.899995,-0.796908,-0.865652,-2.27139,0.90352,-2.76817,1.59489,-1.7355,-2.43541,-0.500126,-2.87785,-1.00394,-0.712698,-0.712698,1.0
3,001a1f289e3bab22_r,3,-1.26624,-1.2648,-1.28114,-1.27033,-1.24794,-1.19769,-0.899995,-0.796908,-0.899995,-0.796908,-0.865652,-2.27139,0.90352,-2.76817,1.59489,-1.7355,-2.43541,-0.500126,-2.87785,-1.00394,-0.712698,-0.712698,1.0
4,001a1f289e3bab22_r,4,-1.20956,-1.20032,-1.22402,-1.22586,-1.18893,-1.14074,-0.899995,-0.796908,-0.899995,-0.796908,-0.865652,-2.27139,0.90352,-2.76817,1.59489,-1.7355,-2.43541,-0.500126,-2.87785,-1.00394,-0.712698,-0.712698,1.0


### Prepare Test Data

In [169]:
xtest = pd.DataFrame(X_test)
ytest = pd.DataFrame(y_test)
print(len(xtest))
print(len(ytest))

143425
143425


In [171]:
test_df = pd.merge(xtest, ytest, left_index=True, right_index=True)

In [173]:
test_df.rename(columns={'0_y':'result','0_x':'match_id'},inplace=True)
test_df.head()

,match_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,result
0,001a1f289e3bab22_b,0,-1.34294,-1.32621,-1.38708,-1.34161,-1.32015,-1.26969,-0.899995,-0.796908,-0.899995,-0.796908,-1.74208,-2.27139,-1.40183,-2.76817,-1.83577,-1.7355,-2.43541,-1.46112,-2.87785,-1.88578,-0.712698,-0.712698,0.0
1,001a1f289e3bab22_b,1,-1.34224,-1.32621,-1.38708,-1.34161,-1.32015,-1.26478,-0.899995,-0.796908,-0.899995,-0.796908,-1.74208,-2.27139,-1.40183,-2.76817,-1.83577,-1.7355,-2.43541,-1.46112,-2.87785,-1.88578,-0.712698,-0.712698,0.0
2,001a1f289e3bab22_b,2,-1.32729,-1.31376,-1.36829,-1.33003,-1.30875,-1.24089,-0.899995,-0.796908,-0.899995,-0.796908,-1.74208,-2.27139,-1.40183,-2.76817,-1.83577,-1.7355,-2.43541,-1.46112,-2.87785,-1.88578,-0.712698,-0.712698,0.0
3,001a1f289e3bab22_b,3,-1.27243,-1.24447,-1.3036,-1.27744,-1.27794,-1.181,-0.899995,-0.796908,-0.899995,-0.796908,-1.74208,-2.27139,-1.40183,-2.76817,-1.83577,-1.7355,-2.43541,-1.46112,-2.87785,-1.88578,-0.712698,-0.712698,0.0
4,001a1f289e3bab22_b,4,-1.21236,-1.18983,-1.23476,-1.21327,-1.21994,-1.13125,-0.899995,-0.796908,-0.899995,-0.796908,-1.74208,-2.27139,-1.40183,-2.76817,-1.83577,-1.7355,-2.43541,-1.46112,-2.87785,-1.88578,-0.712698,-0.712698,0.0


## Parse Game Data to Reformat for Neural Network

In [179]:
def parse_game(grp):
    """Inputs are 1. a groupby group object
    Output is a list of 3d arrays, each element contains 10 rows (which are equivalent to minutes 
       for this dataset). The function tracks the length of each group and stops at length minus 1. 
       After a group is finished the function performs the same process on the next group.
    """  
    t = 10
    start = 0
   
    length = len(grp)

    while t < length:
        
        # take a 10 row chunk and convert it to array
        
        data = grp.iloc[:,2:-1][start:t].values
   
        df_list.append(data)
        result_list.append(grp.iloc[:,-1:][start:t].values[0])
        
        # stop when t reaches length - 1
        start += 1
        t += 1
    
    return df_list  

### Training Data

In [192]:
df_list = []
result_list = []

gb = train_df.groupby('match_id')

gb.apply(lambda group: parse_game(group));

In [193]:
print(len(df_list))
print(len(result_list))

105907
105907


In [194]:
length_r = len(result_list)
y_train = np.array(result_list)
y_train = y_train.ravel()

In [195]:
length = len(df_list)
X_train = np.array(df_list)
X_train = X_train.reshape((length,10,22))

### Test Data

In [184]:
def parse_game_test(grp):
    """Inputs are 1. a groupby group object
    Output is a list of 3d arrays, each element contains 10 rows (which are equivalent to minutes 
       for this dataset). The function tracks the length of each group and stops at length minus 1. 
       After a group is finished the function performs the same process on the next group.
    """  
    t = 10
    start = 0
   
    length = len(grp)

    while t < length:
        
        # take a 10 row chunk and convert it to array
        
        data = grp.iloc[:,2:-1][start:t].values
   
        df_list_test.append(data)
        result_list_test.append(grp.iloc[:,-1:][start:t].values[0])
        
        # stop when t reaches length - 1
        start += 1
        t += 1
    
    return df_list  

In [185]:
df_list_test = []
result_list_test = []

gb = test_df.groupby('match_id')

gb.apply(lambda group: parse_game_test(group));

In [186]:
print(len(df_list_test))
print(len(result_list_test))

105913
105913


In [190]:
length_r = len(result_list_test)
y_test = np.array(result_list_test)
y_test = y_test.ravel()

In [188]:
length = len(df_list_test)
X_test = np.array(df_list_test)
X_test = X_test.reshape((length,10,22))

## Keras LSTM Model

In [68]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

In [191]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(105907, 10, 22)
(105907,)
(105913, 10, 22)
(105913,)


In [200]:
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='mae',
             optimizer='adam',
             metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
105907/105907 [==============================] - 10s - loss: 0.3857 - acc: 0.6205    
Epoch 2/10
105907/105907 [==============================] - 9s - loss: 0.3488 - acc: 0.6542     
Epoch 3/10
105907/105907 [==============================] - 10s - loss: 0.3376 - acc: 0.6650    
Epoch 4/10
105907/105907 [==============================] - 10s - loss: 0.3253 - acc: 0.6785    
Epoch 5/10
105907/105907 [==============================] - 10s - loss: 0.3149 - acc: 0.6884    
Epoch 6/10
105907/105907 [==============================] - 10s - loss: 0.3058 - acc: 0.6974    
Epoch 7/10
105907/105907 [==============================] - 9s - loss: 0.2971 - acc: 0.7060     
Epoch 8/10
105907/105907 [==============================] - 9s - loss: 0.2874 - acc: 0.7164     
Epoch 9/10
105907/105907 [==============================] - 9s - loss: 0.2811 - acc: 0.7227     
Epoch 10/10
105907/105907 [==============================] - 10s - loss: 0.2718 - acc: 0.7321    


In [201]:
score = model.evaluate(X_test, y_test, verbose=0)
score

[0.36678435534292864, 0.63420920945111603]